In [1]:
# !pip install indoxMiner
!pip install opencv-python
!pip install easyocr



[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


  Obtaining dependency information for easyocr from https://files.pythonhosted.org/packages/bb/84/4a2cab0e6adde6a85e7ba543862e5fc0250c51f3ac721a078a55cdcff250/easyocr-1.7.2-py3-none-any.whl.metadata
  Using cached easyocr-1.7.2-py3-none-any.whl.metadata (10 kB)
  Obtaining dependency information for opencv-python-headless from https://files.pythonhosted.org/packages/26/d0/22f68eb23eea053a31655960f133c0be9726c6a881547e6e9e7e2a946c4f/opencv_python_headless-4.10.0.84-cp37-abi3-win_amd64.whl.metadata
  Using cached opencv_python_headless-4.10.0.84-cp37-abi3-win_amd64.whl.metadata (20 kB)
  Obtaining dependency information for scikit-image from https://files.pythonhosted.org/packages/d9/17/b561823143eb931de0f82fed03ae128ef954a9641309602ea0901c357f95/scikit_image-0.24.0-cp312-cp312-win_amd64.whl.metadata
  Using cached scikit_image-0.24.0-cp312-cp312-win_amd64.whl.metadata (14 kB)
  Obtaining dependency information for imageio>=2.33 from https://files.pythonhosted.org/packages/4e/e7/26045404

In [4]:
# Import necessary libraries
import os
import gc
import pandas as pd
from indoxMiner import (
    Extractor,
    ProcessingConfig,
    DocumentProcessor,
    Schema,
    IndoxApi
)

# Initialize Indox API and extractor configuration
indox_api = IndoxApi(api_key='gWLTU2Pei7vfH88SPqTYxZ5vUHUXpu2G')
config = ProcessingConfig(
    ocr_for_images=True,
    ocr_model='easyocr'  # or 'tesseract' or 'paddle'
)

# Initialize the Extractor with the Indox LLM and the Invoice schema
extractor = Extractor(llm=indox_api, schema=Schema.Invoice)

# Define the directory containing images and specify target images
image_directory = r'C:\My Files\IndoxM\IndoxMiner\examples'
target_images = ['invoice_Aaron Bergman_36258-1.png', 'invoice_Aaron Bergman_36259-1.png']  # Add your second image file here

# Build the full paths for each image
image_paths = [os.path.join(image_directory, img) for img in target_images]



In [5]:
# List to hold extraction results for all images
extraction_results = []

# Process and extract data for each image
for image_path in image_paths:
    # Step 1: Create processor instance for each image
    processor = DocumentProcessor([image_path])

    # Step 2: Process document to extract text using OCR
    results = processor.process(config)

    extracted_data = extractor.extract(results)
    extraction_results.append(extracted_data)

    # Clear memory after processing each image
    del processor
    gc.collect()

Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.
Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


In [6]:
# Step 4: Convert extraction results to DataFrame
dataframes = [extractor.to_dataframe(result) for result in extraction_results]
final_df = pd.concat(dataframes, ignore_index=True)

# Display final DataFrame
print("Extracted Data as DataFrame:")
final_df

Extracted Data as DataFrame:


,Invoice Number,Date,Company Name,Address,Customer Name,Items,Subtotal,Tax Amount,Total Amount
0,NaN,2012-03-06,SuperStore,"98103, Seattle, Washington, United States",Aaron Bergman,[{'description': 'Global Push Button Manager's...,48.71,0.0,50.10
1,NaN,2012-03-06,SuperStore,"98103, Seattle, Washington, United States",Aaron Bergman,"[{'description': 'Newell 330', 'quantity': 3, ...",53.82,NaN,58.11


In [7]:
# Convert extraction results to Markdown and Table formats
markdown_output = extractor.to_markdown(extraction_results)
table_output = extractor.to_table(extraction_results)

In [8]:
# Print Markdown and Table outputs
print("\nExtracted Data as Markdown:")
print(markdown_output)


Extracted Data as Markdown:
| items                                                                                                                                                                                                                                                                                                                                                                    |
|--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| [{'Invoice Number': '#36258', 'Date': '2012-03-06', 'Company Name': 'SuperStore', 'Address': '98103, Seattle, Washington, United States', 'Customer Name': 'Aaron Bergman', 'Subtotal': 48.71, 'Tax Amount': 0.0, 'Total Amount': 50.1, 'Items'

In [9]:
print("\nExtracted Data as Table:")
print(table_output)


Extracted Data as Table:
+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
| items                                                                                                                                                                                                                                                                                                                                                                    |
+===================================================================================================================================================================================================================================================